In [1]:
# -*- coding: utf8 -*-
import pandas as pd
import pickle, random
import yaml

In [2]:
millier=1000000
nbMilliers=20
nTiles=5
lexiquePrefix="MGC-160104"
tiragePrefix="MGC-170430-"
echantillon="-%dMo"%nbMilliers
cumul=True

In [3]:
with open("/Users/gilles/Box Sync/2015-Data/"+lexiquePrefix+'-Verbes2.pkl', 'rb') as input:
    lexique = pickle.load(input)

In [4]:
lexique["freqcum"]= (lexique["freq"].cumsum()*1000).astype(int)

In [5]:
rangeTop=lexique["freqcum"].max()

In [6]:
del lexique["ext"]
del lexique["cs"]
del lexique["ms"]
del lexique["vs"]
del lexique["prob"]

In [7]:
%%time
tirage=[]
for n in range(nbMilliers*millier):
    tirage.append(random.randrange(rangeTop))

CPU times: user 37.8 s, sys: 703 ms, total: 38.5 s
Wall time: 38.8 s


In [8]:
lexique["tir1"]=0

In [9]:
%%time
def tirage2triage(tirage,cumul=True):    
    triage=sorted(tirage)
    freqTop=0
    indexMin=0
    tirs={}

    for num,tir in enumerate(triage[:]):
        if tir > freqTop:
            indexMin=lexique[lexique["freqcum"]>=tir][0:1].index.astype(int)[0]
            freqTop=lexique.ix[indexMin,'freqcum']
            tirs[indexMin]=0
        tirs[indexMin] += 1
        if num%500000==0:
            print num,
    print
    if cumul:
        for indexNum in tirs:
            lexique.ix[indexNum,'tir1']+=tirs[indexNum]
    else:
        lexique['tir1']=0
        for indexNum in tirs:
            lexique.ix[indexNum,'tir1']=tirs[indexNum]
        

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 8.11 µs


In [10]:
portion=(nbMilliers*millier)//nTiles
print portion

4000000


In [11]:
for nTile in range(nTiles):
    %time tirage2triage(tirage[nTile*portion:(nTile+1)*portion],cumul)
    print
    print "nTile",nTile
    print lexique[lexique["tir1"]!=0]["tir1"].sum()
    print lexique[lexique["tir1"]!=0]["tir1"].count()
    with open("/Users/gilles/Box Sync/2015-Data/"+tiragePrefix+"%02d"%(nTile)+echantillon+'-Tirage.pkl', 'wb') as output:
        pickle.dump(lexique, output, pickle.HIGHEST_PROTOCOL)

0 500000 1000000 1500000 2000000 2500000 3000000 3500000
CPU times: user 16min 21s, sys: 2min 36s, total: 18min 58s
Wall time: 18min 58s

nTile 0
4000000
60569
0 500000 1000000 1500000 2000000 2500000 3000000 3500000
CPU times: user 16min 20s, sys: 2min 21s, total: 18min 42s
Wall time: 18min 42s

nTile 1
8000000
66583
0 500000 1000000 1500000 2000000 2500000 3000000 3500000
CPU times: user 16min 22s, sys: 2min 24s, total: 18min 47s
Wall time: 18min 48s

nTile 2
12000000
68936
0 500000 1000000 1500000 2000000 2500000 3000000 3500000
CPU times: user 16min 41s, sys: 2min 26s, total: 19min 7s
Wall time: 19min 8s

nTile 3
16000000
70117
0 500000 1000000 1500000 2000000 2500000 3000000 3500000
CPU times: user 18min 10s, sys: 2min 30s, total: 20min 41s
Wall time: 20min 51s

nTile 4
20000000
70824
